In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib import request
import json
# original
pd.options.display.max_columns = 20
pd.options.display.max_rows = 60
# new set
pd.options.display.max_columns = 150
#pd.options.display.max_rows = 2000

In [4]:
df= pd.read_csv(r'dataset_v2.csv',index_col=[0])
df=df.drop(["index"], axis=1)
df.loc[[25,26]]

,Position,Age,Name,Goals,Assists,Non-Penalty Goals,Penalty Kicks Made,Penalty Kicks Attempted,Yellow Cards,Red Cards,xG,npxG,xA,npxG+xA,Shots Total,Shots on target,Shots on target %,Goals/Shot,Goals/Shot on target,Average Shot Distance,Shots from free kicks,npxG/Sh,Goals - xG,Non-Penalty Goals - npxG,Passes Completed,Passes Attempted,Pass Completion %,Total Passing Distance,Progressive Passing Distance,Passes Completed (Short),Passes Attempted (Short),Pass Completion % (Short),Passes Completed (Medium),Passes Attempted (Medium),Pass Completion % (Medium),Passes Completed (Long),Passes Attempted (Long),Pass Completion % (Long),Key Passes,Passes into Final Third,Passes into Penalty Area,Crosses into Penalty Area,Progressive Passes,Live-ball passes,Dead-ball passes,Passes from Free Kicks,Through Balls,Passes Under Pressure,Switches,Crosses,Corner Kicks,Inswinging Corner Kicks,Outswinging Corner Kicks,Straight Corner Kicks,Ground passes,Low Passes,High Passes,Passes Attempted (Left),Passes Attempted (Right),Passes Attempted (Head),Throw-Ins taken,Passes Attempted (Other),Passes Offside,Passes Out of Bounds,Passes Intercepted,Passes Blocked,Shot-Creating Actions,SCA (PassLive),SCA (PassDead),SCA (Drib),SCA (Sh),SCA (Fld),SCA (Def),Goal-Creating Actions,GCA (PassLive),GCA (PassDead),GCA (Drib),GCA (Sh),GCA (Fld),GCA (Def),Tackles,Tackles Won,Tackles (Def 3rd),Tackles (Mid 3rd),Tackles (Att 3rd),Dribblers Tackled,Dribbles Contested,% of dribblers tackled,Dribbled Past,Pressures,Successful Pressures,Successful Pressure %,Pressures (Def 3rd),Pressures (Mid 3rd),Pressures (Att 3rd),Blocks,Shots Blocked,Shots Saved,Interceptions,Tkl+Int,Clearances,Errors,Touches,Touches (Def Pen),Touches (Def 3rd),Touches (Mid 3rd),Touches (Att 3rd),Touches (Att Pen),Touches (Live-Ball),Dribbles Completed,Dribbles Attempted,Successful Dribble %,Players Dribbled Past,Nutmegs,Carries,Total Carrying Distance,Progressive Carrying Distance,Progressive Carries,Carries into Final Third,Carries into Penalty Area,Miscontrols,Dispossessed,Pass Targets,Passes Received,Passes Received %,Progressive Passes Rec,Second Yellow Card,Fouls Committed,Fouls Drawn,Offsides,Penalty Kicks Won,Penalty Kicks Conceded,Own Goals,Ball Recoveries,Aerials won,Aerials lost,% of Aerials Won,Market value
25,Centre-Back,35,Adil Rami,0.18,0.00,0.18,0.00,0.00,0.00,0.0,0.04,0.04,0.00,0.04,0.37,0.18,50.00%,0.50,1.00,6.9,0.00,0.08,0.15,0.15,26.93,32.27,83.40%,578.18,231.82,9.59,10.33,92.90%,11.43,12.73,89.90%,5.35,8.11,65.90%,0.00,1.66,0.00,0.00,1.66,30.80,1.48,1.11,0.00,2.77,2.03,0.00,0.00,0.00,0.00,0.00,22.87,3.5,5.90,3.87,26.56,0.55,0.37,0.00,0.00,1.48,0.74,0.74,0.37,0.37,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.92,0.55,0.18,0.55,0.18,0.74,0.92,80.00%,0.18,5.16,2.58,50.00%,2.58,2.40,0.18,0.92,0.18,0.0,1.48,2.40,3.14,0.0,39.10,5.53,24.16,15.12,2.58,0.92,37.81,0.18,0.18,100.00%,0.18,0.00,22.13,121.35,62.34,1.11,0.55,0.00,0.18,0.37,26.93,25.27,93.80%,0.00,0.0,0.74,0.55,0.00,0.00,0.0,0.0,5.53,1.84,0.55,76.90%,"€250Th. Last update: May 21, 2021"
26,Right Winger,26,Adnan Januzaj,0.17,0.25,0.04,0.12,0.12,0.25,0.0,0.25,0.15,0.31,0.46,2.24,0.54,24.10%,0.02,0.08,21.7,0.37,0.07,-0.08,-0.11,30.15,44.40,67.90%,607.66,230.79,13.41,16.07,83.50%,10.76,14.37,74.90%,5.44,11.88,45.80%,2.53,2.62,2.62,0.62,5.03,36.34,8.06,2.58,0.37,7.48,3.49,3.49,4.73,2.04,1.91,0.21,28.12,5.4,10.88,37.21,5.61,0.46,0.62,0.04,0.33,1.58,1.83,0.87,4.49,2.24,1.5,0.33,0.17,0.25,0.0,0.62,0.29,0.12,0.04,0.08,0.08,0.0,1.04,0.54,0.25,0.58,0.21,0.21,0.79,26.30%,0.58,10.26,2.82,27.50%,1.66,5.23,3.36,1.08,0.21,0.0,0.75,1.79,0.66,0.0,61.14,0.91,5.11,28.86,32.10,3.53,53.91,3.45,5.19,66.40%,3.70,0.37,47.76,229.92,139.76,8.60,2.74,1.04,3.53,3.24,53.37,44.52,83.40%,6.44,0.0,0.96,3.86,0.21,0.04,0.0,0.0,4.94,0.71,1.00,41.50%,€12.00


# Step 1: Refine position
- For now we have many position
- We want to group all position into 3 category
1. FW (Forward)
2. MD (Midfielder)
3. DF (Defender)
- Note that we dropped all goalkeepers becuase their football statistics mertics are different

In [236]:
unique_position = list(df["Position"].unique())
unique_position

['Centre-Forward',
 'Left-Back',
 'Central Midfield',
 'Right-Back',
 'Centre-Back',
 'Defensive Midfield',
 'Attacking Midfield',
 'Right Midfield',
 'Right Winger',
 'Left Winger',
 'Second Striker',
 'Left Midfield']

In [259]:
change_pos_dict = {}
fw_lst = ['Forward','Winger','Striker' ]
for pos_name in unique_position:
    if 'Midfield' in pos_name:
        change_pos_dict[pos_name] = "Midfielder"
    elif any(name in pos_name for name in fw_lst ):
        change_pos_dict[pos_name] = "Forward"
    else:
        change_pos_dict[pos_name] = "Defender"
change_pos_dict

{'Centre-Forward': 'Forward',
 'Left-Back': 'Defender',
 'Central Midfield': 'Midfielder',
 'Right-Back': 'Defender',
 'Centre-Back': 'Defender',
 'Defensive Midfield': 'Midfielder',
 'Attacking Midfield': 'Midfielder',
 'Right Midfield': 'Midfielder',
 'Right Winger': 'Forward',
 'Left Winger': 'Forward',
 'Second Striker': 'Forward',
 'Left Midfield': 'Midfielder'}

In [238]:
df["Position"].replace(change_pos_dict, inplace=True)
df

,Position,Age,Name,Goals,Assists,Non-Penalty Goals,Penalty Kicks Made,Penalty Kicks Attempted,Yellow Cards,Red Cards,xG,npxG,xA,npxG+xA,Shots Total,Shots on target,Shots on target %,Goals/Shot,Goals/Shot on target,Average Shot Distance,Shots from free kicks,npxG/Sh,Goals - xG,Non-Penalty Goals - npxG,Passes Completed,Passes Attempted,Pass Completion %,Total Passing Distance,Progressive Passing Distance,Passes Completed (Short),Passes Attempted (Short),Pass Completion % (Short),Passes Completed (Medium),Passes Attempted (Medium),Pass Completion % (Medium),Passes Completed (Long),Passes Attempted (Long),Pass Completion % (Long),Key Passes,Passes into Final Third,Passes into Penalty Area,Crosses into Penalty Area,Progressive Passes,Live-ball passes,Dead-ball passes,Passes from Free Kicks,Through Balls,Passes Under Pressure,Switches,Crosses,...,Dribbled Past,Pressures,Successful Pressures,Successful Pressure %,Pressures (Def 3rd),Pressures (Mid 3rd),Pressures (Att 3rd),Blocks,Shots Blocked,Shots Saved,Interceptions,Tkl+Int,Clearances,Errors,Touches,Touches (Def Pen),Touches (Def 3rd),Touches (Mid 3rd),Touches (Att 3rd),Touches (Att Pen),Touches (Live-Ball),Dribbles Completed,Dribbles Attempted,Successful Dribble %,Players Dribbled Past,Nutmegs,Carries,Total Carrying Distance,Progressive Carrying Distance,Progressive Carries,Carries into Final Third,Carries into Penalty Area,Miscontrols,Dispossessed,Pass Targets,Passes Received,Passes Received %,Progressive Passes Rec,Second Yellow Card,Fouls Committed,Fouls Drawn,Offsides,Penalty Kicks Won,Penalty Kicks Conceded,Own Goals,Ball Recoveries,Aerials won,Aerials lost,% of Aerials Won,Market value
0,Foward,21,Aaron Connolly,0.19,0.00,0.19,0.00,0.00,0.00,0.00,0.54,0.54,0.08,0.61,3.06,0.96,31.30%,0.06,0.20,13.4,0.00,0.17,-0.34,-0.34,8.23,10.53,78.20%,113.94,8.81,5.17,6.70,77.10%,2.68,3.06,87.50%,0.19,0.19,100.00%,0.57,0.00,0.00,0.00,0.00,9.77,0.77,0.00,0.00,1.91,0.00,0.19,...,0.57,13.40,4.21,31.40%,0.77,6.51,6.13,1.91,0.19,0.0,0.00,0.96,0.19,0.0,22.40,0.19,1.91,9.00,13.02,4.40,21.64,0.38,0.57,66.70%,0.38,0.00,11.87,71.81,41.94,2.30,0.96,0.57,2.49,0.96,40.98,15.13,36.90%,6.70,0.0,0.38,2.30,0.19,0.00,0.0,0.00,2.87,0.96,3.26,22.70%,€7.00
1,Defender,31,Aaron Cresswell,0.02,0.20,0.02,0.00,0.00,0.07,0.00,0.03,0.03,0.16,0.20,0.62,0.10,16.00%,0.04,0.25,22.8,0.32,0.05,-0.01,-0.01,49.11,63.91,76.80%,929.47,423.76,21.52,23.98,89.70%,20.24,23.80,85.00%,6.67,13.81,48.30%,1.67,4.11,0.96,0.37,4.68,50.71,13.19,2.26,0.02,5.49,1.87,2.41,...,0.52,6.35,2.14,33.70%,3.27,2.46,0.62,1.43,0.42,0.0,1.21,2.09,2.49,0.0,71.27,4.28,21.29,32.42,21.64,0.76,58.05,0.15,0.44,33.30%,0.15,0.00,40.64,185.56,94.16,3.64,1.62,0.12,0.27,0.32,44.43,42.61,95.90%,1.06,0.0,0.52,0.44,0.02,0.00,0.0,0.00,8.49,1.16,0.69,62.70%,€5.00
2,Defender,19,Aaron Hickey,0.21,0.00,0.21,0.00,0.00,0.28,0.07,0.03,0.03,0.02,0.05,0.77,0.42,54.50%,0.27,0.50,23.6,0.00,0.03,0.18,0.18,37.27,44.37,84.00%,597.47,155.69,19.02,20.69,91.90%,15.33,18.18,84.30%,2.37,4.18,56.70%,0.49,1.18,0.28,0.07,1.81,35.67,8.71,0.98,0.00,6.76,0.42,1.18,...,0.35,13.37,3.69,27.60%,6.41,4.32,2.65,2.30,0.56,0.0,1.04,2.51,1.60,0.0,55.10,3.07,17.69,24.80,16.02,0.84,46.74,1.32,2.09,63.30%,1.53,0.07,35.94,196.58,106.93,4.74,0.91,0.35,0.91,0.84,37.89,33.37,88.10%,2.51,0.0,1.32,2.23,0.00,0.00,0.0,0.00,5.99,0.63,0.84,42.90%,€8.00
3,Foward,24,Aaron Leya Iseka,0.31,0.00,0.21,0.10,0.21,0.42,0.00,0.35,0.20,0.07,0.27,1.56,0.52,33.30%,0.13,0.40,15.1,0.00,0.13,-0.04,0.01,13.53,17.90,75.60%,207.57,34.65,7.70,9.36,82.20%,3.95,5.41,73.10%,1.25,1.66,75.00%,1.14,1.04,0.31,0.00,0.94,17.48,0.42,0.00,0.31,5.51,0.21,0.31,...,0.73,9.57,2.29,23.90%,0.31,4.27,4.99,0.52,0.31,0.0,0.21,0.94,0.42,0.0,29.13,1.14,3.33,16.65,11.45,2.71,28.51,1.04,1.98,52.60%,1.25,0.00,18.94,77.62,37.98,2.81,1.14,0.83,3.23,1.98,48.90,23.10,47.20%,5.51,0.0,2.39,1.14,0.21,0.00,0.0,0.00,3.12,2.60,3.43,43.10%,€2.20
4,Defender,24,AarÃ³n MartÃ­n,0.00,0.11,0.00,0.00,0.00,0.11,0.00,0.02,0.02,0.10,0.11,0.54,0.07,13.30%,0.00,0.00,23.1,0.04,0.0

# Here we inspect what kinds of statsitical/numerical value is "object" (i.e. string)

In [239]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1894 entries, 0 to 1893
Data columns (total 138 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Position                       1894 non-null   object 
 1   Age                            1894 non-null   int64  
 2   Name                           1894 non-null   object 
 3   Goals                          1894 non-null   float64
 4   Assists                        1894 non-null   float64
 5   Non-Penalty Goals              1894 non-null   float64
 6   Penalty Kicks Made             1894 non-null   float64
 7   Penalty Kicks Attempted        1894 non-null   float64
 8   Yellow Cards                   1894 non-null   float64
 9   Red Cards                      1894 non-null   float64
 10  xG                             1894 non-null   float64
 11  npxG                           1894 non-null   float64
 12  xA                             1894 non-null   

In [241]:
# original
pd.options.display.max_columns = 20
pd.options.display.max_rows = 60
# new set
pd.options.display.max_columns = 100
#pd.options.display.max_rows = 2000



# Covert content from % to decimal

In [242]:
for i in df.columns:
    if "%" in i:
        df[i] = df[i].str.replace('%','')
        print(df.loc[[0]][i])

0    31.30
Name: Shots on target %, dtype: object
0    78.20
Name: Pass Completion %, dtype: object
0    77.10
Name: Pass Completion % (Short), dtype: object
0    87.50
Name: Pass Completion % (Medium), dtype: object
0    100.00
Name: Pass Completion % (Long), dtype: object
0    0.00
Name: % of dribblers tackled, dtype: object
0    31.40
Name: Successful Pressure %, dtype: object
0    66.70
Name: Successful Dribble %, dtype: object
0    36.90
Name: Passes Received %, dtype: object
0    22.70
Name: % of Aerials Won, dtype: object


# Covert from object to float64

In [243]:
for i in df.columns:
    if "%" in i:
        df[i] = pd.to_numeric(df[i])
        df[i] = df[i]/100
        print(df.loc[[0]][i])

df.to_csv("dataset_v3.csv")

0    0.313
Name: Shots on target %, dtype: float64
0    0.782
Name: Pass Completion %, dtype: float64
0    0.771
Name: Pass Completion % (Short), dtype: float64
0    0.875
Name: Pass Completion % (Medium), dtype: float64
0    1.0
Name: Pass Completion % (Long), dtype: float64
0    0.0
Name: % of dribblers tackled, dtype: float64
0    0.314
Name: Successful Pressure %, dtype: float64
0    0.667
Name: Successful Dribble %, dtype: float64
0    0.369
Name: Passes Received %, dtype: float64
0    0.227
Name: % of Aerials Won, dtype: float64


# Step 2: Clean the market value column

# v4: Before converting market value

In [244]:
df=pd.read_csv("dataset_v3.csv",index_col=[0])
df.loc[[37]]


,Position,Age,Name,Goals,Assists,Non-Penalty Goals,Penalty Kicks Made,Penalty Kicks Attempted,Yellow Cards,Red Cards,xG,npxG,xA,npxG+xA,Shots Total,Shots on target,Shots on target %,Goals/Shot,Goals/Shot on target,Average Shot Distance,Shots from free kicks,npxG/Sh,Goals - xG,Non-Penalty Goals - npxG,Passes Completed,Passes Attempted,Pass Completion %,Total Passing Distance,Progressive Passing Distance,Passes Completed (Short),Passes Attempted (Short),Pass Completion % (Short),Passes Completed (Medium),Passes Attempted (Medium),Pass Completion % (Medium),Passes Completed (Long),Passes Attempted (Long),Pass Completion % (Long),Key Passes,Passes into Final Third,Passes into Penalty Area,Crosses into Penalty Area,Progressive Passes,Live-ball passes,Dead-ball passes,Passes from Free Kicks,Through Balls,Passes Under Pressure,Switches,Crosses,...,Dribbled Past,Pressures,Successful Pressures,Successful Pressure %,Pressures (Def 3rd),Pressures (Mid 3rd),Pressures (Att 3rd),Blocks,Shots Blocked,Shots Saved,Interceptions,Tkl+Int,Clearances,Errors,Touches,Touches (Def Pen),Touches (Def 3rd),Touches (Mid 3rd),Touches (Att 3rd),Touches (Att Pen),Touches (Live-Ball),Dribbles Completed,Dribbles Attempted,Successful Dribble %,Players Dribbled Past,Nutmegs,Carries,Total Carrying Distance,Progressive Carrying Distance,Progressive Carries,Carries into Final Third,Carries into Penalty Area,Miscontrols,Dispossessed,Pass Targets,Passes Received,Passes Received %,Progressive Passes Rec,Second Yellow Card,Fouls Committed,Fouls Drawn,Offsides,Penalty Kicks Won,Penalty Kicks Conceded,Own Goals,Ball Recoveries,Aerials won,Aerials lost,% of Aerials Won,Market value
37,Defender,34,Ahmed Elmohamady,0.0,0.0,0.0,0.0,0.0,0.22,0.0,0.06,0.06,0.07,0.12,1.01,0.11,0.111,0.0,0.0,11.2,0.0,0.05,-0.06,-0.06,48.09,61.8,0.778,925.96,388.99,20.11,21.8,0.923,20.56,26.52,0.775,6.97,12.13,0.574,1.35,3.82,2.81,0.9,5.28,49.55,12.25,1.69,0.0,7.42,1.57,3.26,...,0.22,7.87,3.03,0.386,4.04,2.81,1.01,1.57,0.45,0.0,3.03,4.94,3.6,0.0,71.91,4.72,23.6,29.78,21.91,1.24,60.0,0.45,0.56,0.8,0.45,0.0,36.29,164.94,92.02,4.72,1.69,0.22,0.11,0.79,41.57,38.88,0.935,1.8,0.0,0.79,0.22,0.0,0.0,0.0,0.0,9.66,3.03,1.01,0.75,"€800Th. Last update: Oct 28, 2021"


In [245]:
df.loc[[36]]

,Position,Age,Name,Goals,Assists,Non-Penalty Goals,Penalty Kicks Made,Penalty Kicks Attempted,Yellow Cards,Red Cards,xG,npxG,xA,npxG+xA,Shots Total,Shots on target,Shots on target %,Goals/Shot,Goals/Shot on target,Average Shot Distance,Shots from free kicks,npxG/Sh,Goals - xG,Non-Penalty Goals - npxG,Passes Completed,Passes Attempted,Pass Completion %,Total Passing Distance,Progressive Passing Distance,Passes Completed (Short),Passes Attempted (Short),Pass Completion % (Short),Passes Completed (Medium),Passes Attempted (Medium),Pass Completion % (Medium),Passes Completed (Long),Passes Attempted (Long),Pass Completion % (Long),Key Passes,Passes into Final Third,Passes into Penalty Area,Crosses into Penalty Area,Progressive Passes,Live-ball passes,Dead-ball passes,Passes from Free Kicks,Through Balls,Passes Under Pressure,Switches,Crosses,...,Dribbled Past,Pressures,Successful Pressures,Successful Pressure %,Pressures (Def 3rd),Pressures (Mid 3rd),Pressures (Att 3rd),Blocks,Shots Blocked,Shots Saved,Interceptions,Tkl+Int,Clearances,Errors,Touches,Touches (Def Pen),Touches (Def 3rd),Touches (Mid 3rd),Touches (Att 3rd),Touches (Att Pen),Touches (Live-Ball),Dribbles Completed,Dribbles Attempted,Successful Dribble %,Players Dribbled Past,Nutmegs,Carries,Total Carrying Distance,Progressive Carrying Distance,Progressive Carries,Carries into Final Third,Carries into Penalty Area,Miscontrols,Dispossessed,Pass Targets,Passes Received,Passes Received %,Progressive Passes Rec,Second Yellow Card,Fouls Committed,Fouls Drawn,Offsides,Penalty Kicks Won,Penalty Kicks Conceded,Own Goals,Ball Recoveries,Aerials won,Aerials lost,% of Aerials Won,Market value
36,Midfielder,29,Ahmad Benali,0.0,0.11,0.0,0.0,0.0,0.11,0.0,0.07,0.07,0.1,0.15,0.77,0.22,0.286,0.0,0.0,16.8,0.0,0.08,-0.07,-0.07,46.04,54.29,0.848,830.55,220.88,19.23,21.21,0.907,20.44,22.86,0.894,4.84,7.8,0.62,1.21,3.52,0.55,0.11,3.74,50.11,4.18,1.98,0.0,8.46,1.54,1.32,...,2.64,20.55,5.6,0.273,8.24,8.57,3.74,0.99,0.11,0.0,0.77,2.31,1.54,0.0,62.97,3.96,17.58,35.38,13.19,2.09,58.9,0.66,0.99,0.667,0.66,0.0,43.19,181.43,78.24,3.3,1.32,0.11,0.99,1.65,50.66,45.05,0.889,3.3,0.0,0.88,1.1,0.11,0.0,0.0,0.0,8.13,0.99,0.44,0.692,€1.50


In [246]:
import re
df.loc[[37]]
df.to_csv("dataset_v4.csv")

# v5: Coverted thousandas only

In [247]:
def covert_thousands(x):
    
    deleted_last = re.sub(r' Last update.*','', x)
    thousands = re.sub(r'Th.','', deleted_last)
    thousands_nosym=re.sub(r'€','',thousands)
    value = float(thousands_nosym)/1000
    return value


In [248]:
def check_need_clean(x):
    if "Th" in x:
        return True
    else:
        return False
    

In [249]:
df['Market value'] =  df['Market value'].apply(lambda x: covert_thousands(x) if check_need_clean(x) else  x)
df.loc[[37]]


,Position,Age,Name,Goals,Assists,Non-Penalty Goals,Penalty Kicks Made,Penalty Kicks Attempted,Yellow Cards,Red Cards,xG,npxG,xA,npxG+xA,Shots Total,Shots on target,Shots on target %,Goals/Shot,Goals/Shot on target,Average Shot Distance,Shots from free kicks,npxG/Sh,Goals - xG,Non-Penalty Goals - npxG,Passes Completed,Passes Attempted,Pass Completion %,Total Passing Distance,Progressive Passing Distance,Passes Completed (Short),Passes Attempted (Short),Pass Completion % (Short),Passes Completed (Medium),Passes Attempted (Medium),Pass Completion % (Medium),Passes Completed (Long),Passes Attempted (Long),Pass Completion % (Long),Key Passes,Passes into Final Third,Passes into Penalty Area,Crosses into Penalty Area,Progressive Passes,Live-ball passes,Dead-ball passes,Passes from Free Kicks,Through Balls,Passes Under Pressure,Switches,Crosses,...,Dribbled Past,Pressures,Successful Pressures,Successful Pressure %,Pressures (Def 3rd),Pressures (Mid 3rd),Pressures (Att 3rd),Blocks,Shots Blocked,Shots Saved,Interceptions,Tkl+Int,Clearances,Errors,Touches,Touches (Def Pen),Touches (Def 3rd),Touches (Mid 3rd),Touches (Att 3rd),Touches (Att Pen),Touches (Live-Ball),Dribbles Completed,Dribbles Attempted,Successful Dribble %,Players Dribbled Past,Nutmegs,Carries,Total Carrying Distance,Progressive Carrying Distance,Progressive Carries,Carries into Final Third,Carries into Penalty Area,Miscontrols,Dispossessed,Pass Targets,Passes Received,Passes Received %,Progressive Passes Rec,Second Yellow Card,Fouls Committed,Fouls Drawn,Offsides,Penalty Kicks Won,Penalty Kicks Conceded,Own Goals,Ball Recoveries,Aerials won,Aerials lost,% of Aerials Won,Market value
37,Defender,34,Ahmed Elmohamady,0.0,0.0,0.0,0.0,0.0,0.22,0.0,0.06,0.06,0.07,0.12,1.01,0.11,0.111,0.0,0.0,11.2,0.0,0.05,-0.06,-0.06,48.09,61.8,0.778,925.96,388.99,20.11,21.8,0.923,20.56,26.52,0.775,6.97,12.13,0.574,1.35,3.82,2.81,0.9,5.28,49.55,12.25,1.69,0.0,7.42,1.57,3.26,...,0.22,7.87,3.03,0.386,4.04,2.81,1.01,1.57,0.45,0.0,3.03,4.94,3.6,0.0,71.91,4.72,23.6,29.78,21.91,1.24,60.0,0.45,0.56,0.8,0.45,0.0,36.29,164.94,92.02,4.72,1.69,0.22,0.11,0.79,41.57,38.88,0.935,1.8,0.0,0.79,0.22,0.0,0.0,0.0,0.0,9.66,3.03,1.01,0.75,0.8


In [250]:
df.to_csv("dataset_v5.csv")


# v6: Coverted all martket value

In [251]:
def check_sign(x):
    if isinstance(x,float):
        return False
    else:
        return True

In [252]:


df['Market value'] =  df['Market value'].apply(lambda x:(re.sub(r'€','',x) if check_sign(x) else x))
df.loc[[36]]
df.to_csv("dataset_v6.csv")


In [261]:
df

,Age,Foward,Midfielder,Defender,Goals,Assists,Non-Penalty Goals,Penalty Kicks Made,Penalty Kicks Attempted,Yellow Cards,Red Cards,xG,npxG,xA,npxG+xA,Shots Total,Shots on target,Shots on target %,Goals/Shot,Goals/Shot on target,Average Shot Distance,Shots from free kicks,npxG/Sh,Goals - xG,Non-Penalty Goals - npxG,Passes Completed,Passes Attempted,Pass Completion %,Total Passing Distance,Progressive Passing Distance,Passes Completed (Short),Passes Attempted (Short),Pass Completion % (Short),Passes Completed (Medium),Passes Attempted (Medium),Pass Completion % (Medium),Passes Completed (Long),Passes Attempted (Long),Pass Completion % (Long),Key Passes,Passes into Final Third,Passes into Penalty Area,Crosses into Penalty Area,Progressive Passes,Live-ball passes,Dead-ball passes,Passes from Free Kicks,Through Balls,Passes Under Pressure,Switches,...,Dribbled Past,Pressures,Successful Pressures,Successful Pressure %,Pressures (Def 3rd),Pressures (Mid 3rd),Pressures (Att 3rd),Blocks,Shots Blocked,Shots Saved,Interceptions,Tkl+Int,Clearances,Errors,Touches,Touches (Def Pen),Touches (Def 3rd),Touches (Mid 3rd),Touches (Att 3rd),Touches (Att Pen),Touches (Live-Ball),Dribbles Completed,Dribbles Attempted,Successful Dribble %,Players Dribbled Past,Nutmegs,Carries,Total Carrying Distance,Progressive Carrying Distance,Progressive Carries,Carries into Final Third,Carries into Penalty Area,Miscontrols,Dispossessed,Pass Targets,Passes Received,Passes Received %,Progressive Passes Rec,Second Yellow Card,Fouls Committed,Fouls Drawn,Offsides,Penalty Kicks Won,Penalty Kicks Conceded,Own Goals,Ball Recoveries,Aerials won,Aerials lost,% of Aerials Won,Market value
0,21,1,0,0,0.19,0.00,0.19,0.00,0.00,0.00,0.00,0.54,0.54,0.08,0.61,3.06,0.96,0.313,0.06,0.20,13.4,0.00,0.17,-0.34,-0.34,8.23,10.53,0.782,113.94,8.81,5.17,6.70,0.771,2.68,3.06,0.875,0.19,0.19,1.000,0.57,0.00,0.00,0.00,0.00,9.77,0.77,0.00,0.00,1.91,0.00,...,0.57,13.40,4.21,0.314,0.77,6.51,6.13,1.91,0.19,0.0,0.00,0.96,0.19,0.0,22.40,0.19,1.91,9.00,13.02,4.40,21.64,0.38,0.57,0.667,0.38,0.00,11.87,71.81,41.94,2.30,0.96,0.57,2.49,0.96,40.98,15.13,0.369,6.70,0.0,0.38,2.30,0.19,0.00,0.0,0.00,2.87,0.96,3.26,0.227,7.0
1,31,0,0,1,0.02,0.20,0.02,0.00,0.00,0.07,0.00,0.03,0.03,0.16,0.20,0.62,0.10,0.160,0.04,0.25,22.8,0.32,0.05,-0.01,-0.01,49.11,63.91,0.768,929.47,423.76,21.52,23.98,0.897,20.24,23.80,0.850,6.67,13.81,0.483,1.67,4.11,0.96,0.37,4.68,50.71,13.19,2.26,0.02,5.49,1.87,...,0.52,6.35,2.14,0.337,3.27,2.46,0.62,1.43,0.42,0.0,1.21,2.09,2.49,0.0,71.27,4.28,21.29,32.42,21.64,0.76,58.05,0.15,0.44,0.333,0.15,0.00,40.64,185.56,94.16,3.64,1.62,0.12,0.27,0.32,44.43,42.61,0.959,1.06,0.0,0.52,0.44,0.02,0.00,0.0,0.00,8.49,1.16,0.69,0.627,5.0
2,19,0,0,1,0.21,0.00,0.21,0.00,0.00,0.28,0.07,0.03,0.03,0.02,0.05,0.77,0.42,0.545,0.27,0.50,23.6,0.00,0.03,0.18,0.18,37.27,44.37,0.840,597.47,155.69,19.02,20.69,0.919,15.33,18.18,0.843,2.37,4.18,0.567,0.49,1.18,0.28,0.07,1.81,35.67,8.71,0.98,0.00,6.76,0.42,...,0.35,13.37,3.69,0.276,6.41,4.32,2.65,2.30,0.56,0.0,1.04,2.51,1.60,0.0,55.10,3.07,17.69,24.80,16.02,0.84,46.74,1.32,2.09,0.633,1.53,0.07,35.94,196.58,106.93,4.74,0.91,0.35,0.91,0.84,37.89,33.37,0.881,2.51,0.0,1.32,2.23,0.00,0.00,0.0,0.00,5.99,0.63,0.84,0.429,8.0
3,24,1,0,0,0.31,0.00,0.21,0.10,0.21,0.42,0.00,0.35,0.20,0.07,0.27,1.56,0.52,0.333,0.13,0.40,15.1,0.00,0.13,-0.04,0.01,13.53,17.90,0.756,207.57,34.65,7.70,9.36,0.822,3.95,5.41,0.731,1.25,1.66,0.750,1.14,1.04,0.31,0.00,0.94,17.48,0.42,0.00,0.31,5.51,0.21,...,0.73,9.57,2.29,0.239,0.31,4.27,4.99,0.52,0.31,0.0,0.21,0.94,0.42,0.0,29.13,1.14,3.33,16.65,11.45,2.71,28.51,1.04,1.98,0.526,1.25,0.00,18.94,77.62,37.98,2.81,1.14,0.83,3.23,1.98,48.90,23.10,0.472,5.51,0.0,2.39,1.14,0.21,0.00,0.0,0.00,3.12,2.60,3.43,0.431,2.2
4,24,0,0,1,0.00,0.11,0.00,0.00,0.00,0.11,0.00,0.02,0.02,0.10,0.11,0.54,0.07,0.133,0.00,0.00,23.1,0.04,0.03,-0.02,-0.02,37.26,48.72,0.765,656.88,231.19,17.70,19.59,0.904,14.69,18.07,0.813,4.06,9.29,0.438,1.02,1.96,0.65,0.44,2.50,38.31,10.41,1.20,0.07,4.4

# v7: Coverted all martket value from object to float 64

In [254]:
df["Market value"]= pd.to_numeric(df["Market value"])
df.info(verbose=True, null_counts=True)
df.to_csv("dataset_v7.csv")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1894 entries, 0 to 1893
Data columns (total 138 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Position                       1894 non-null   object 
 1   Age                            1894 non-null   int64  
 2   Name                           1894 non-null   object 
 3   Goals                          1894 non-null   float64
 4   Assists                        1894 non-null   float64
 5   Non-Penalty Goals              1894 non-null   float64
 6   Penalty Kicks Made             1894 non-null   float64
 7   Penalty Kicks Attempted        1894 non-null   float64
 8   Yellow Cards                   1894 non-null   float64
 9   Red Cards                      1894 non-null   float64
 10  xG                             1894 non-null   float64
 11  npxG                           1894 non-null   float64
 12  xA                             1894 non-null   

# v8: One-shot encoding for position

In [255]:
df=pd.read_csv(r"dataset_v7.csv",index_col=[0])
one_hot = pd.get_dummies(df['Position'])
df = df.drop('Position',axis = 1)
df = df.join(one_hot)
df.to_csv("dataset_v8.csv")
df

,Age,Name,Goals,Assists,Non-Penalty Goals,Penalty Kicks Made,Penalty Kicks Attempted,Yellow Cards,Red Cards,xG,npxG,xA,npxG+xA,Shots Total,Shots on target,Shots on target %,Goals/Shot,Goals/Shot on target,Average Shot Distance,Shots from free kicks,npxG/Sh,Goals - xG,Non-Penalty Goals - npxG,Passes Completed,Passes Attempted,Pass Completion %,Total Passing Distance,Progressive Passing Distance,Passes Completed (Short),Passes Attempted (Short),Pass Completion % (Short),Passes Completed (Medium),Passes Attempted (Medium),Pass Completion % (Medium),Passes Completed (Long),Passes Attempted (Long),Pass Completion % (Long),Key Passes,Passes into Final Third,Passes into Penalty Area,Crosses into Penalty Area,Progressive Passes,Live-ball passes,Dead-ball passes,Passes from Free Kicks,Through Balls,Passes Under Pressure,Switches,Crosses,Corner Kicks,...,Successful Pressure %,Pressures (Def 3rd),Pressures (Mid 3rd),Pressures (Att 3rd),Blocks,Shots Blocked,Shots Saved,Interceptions,Tkl+Int,Clearances,Errors,Touches,Touches (Def Pen),Touches (Def 3rd),Touches (Mid 3rd),Touches (Att 3rd),Touches (Att Pen),Touches (Live-Ball),Dribbles Completed,Dribbles Attempted,Successful Dribble %,Players Dribbled Past,Nutmegs,Carries,Total Carrying Distance,Progressive Carrying Distance,Progressive Carries,Carries into Final Third,Carries into Penalty Area,Miscontrols,Dispossessed,Pass Targets,Passes Received,Passes Received %,Progressive Passes Rec,Second Yellow Card,Fouls Committed,Fouls Drawn,Offsides,Penalty Kicks Won,Penalty Kicks Conceded,Own Goals,Ball Recoveries,Aerials won,Aerials lost,% of Aerials Won,Market value,Defender,Foward,Midfielder
0,21,Aaron Connolly,0.19,0.00,0.19,0.00,0.00,0.00,0.00,0.54,0.54,0.08,0.61,3.06,0.96,0.313,0.06,0.20,13.4,0.00,0.17,-0.34,-0.34,8.23,10.53,0.782,113.94,8.81,5.17,6.70,0.771,2.68,3.06,0.875,0.19,0.19,1.000,0.57,0.00,0.00,0.00,0.00,9.77,0.77,0.00,0.00,1.91,0.00,0.19,0.00,...,0.314,0.77,6.51,6.13,1.91,0.19,0.0,0.00,0.96,0.19,0.0,22.40,0.19,1.91,9.00,13.02,4.40,21.64,0.38,0.57,0.667,0.38,0.00,11.87,71.81,41.94,2.30,0.96,0.57,2.49,0.96,40.98,15.13,0.369,6.70,0.0,0.38,2.30,0.19,0.00,0.0,0.00,2.87,0.96,3.26,0.227,7.0,0,1,0
1,31,Aaron Cresswell,0.02,0.20,0.02,0.00,0.00,0.07,0.00,0.03,0.03,0.16,0.20,0.62,0.10,0.160,0.04,0.25,22.8,0.32,0.05,-0.01,-0.01,49.11,63.91,0.768,929.47,423.76,21.52,23.98,0.897,20.24,23.80,0.850,6.67,13.81,0.483,1.67,4.11,0.96,0.37,4.68,50.71,13.19,2.26,0.02,5.49,1.87,2.41,2.56,...,0.337,3.27,2.46,0.62,1.43,0.42,0.0,1.21,2.09,2.49,0.0,71.27,4.28,21.29,32.42,21.64,0.76,58.05,0.15,0.44,0.333,0.15,0.00,40.64,185.56,94.16,3.64,1.62,0.12,0.27,0.32,44.43,42.61,0.959,1.06,0.0,0.52,0.44,0.02,0.00,0.0,0.00,8.49,1.16,0.69,0.627,5.0,1,0,0
2,19,Aaron Hickey,0.21,0.00,0.21,0.00,0.00,0.28,0.07,0.03,0.03,0.02,0.05,0.77,0.42,0.545,0.27,0.50,23.6,0.00,0.03,0.18,0.18,37.27,44.37,0.840,597.47,155.69,19.02,20.69,0.919,15.33,18.18,0.843,2.37,4.18,0.567,0.49,1.18,0.28,0.07,1.81,35.67,8.71,0.98,0.00,6.76,0.42,1.18,0.00,...,0.276,6.41,4.32,2.65,2.30,0.56,0.0,1.04,2.51,1.60,0.0,55.10,3.07,17.69,24.80,16.02,0.84,46.74,1.32,2.09,0.633,1.53,0.07,35.94,196.58,106.93,4.74,0.91,0.35,0.91,0.84,37.89,33.37,0.881,2.51,0.0,1.32,2.23,0.00,0.00,0.0,0.00,5.99,0.63,0.84,0.429,8.0,1,0,0
3,24,Aaron Leya Iseka,0.31,0.00,0.21,0.10,0.21,0.42,0.00,0.35,0.20,0.07,0.27,1.56,0.52,0.333,0.13,0.40,15.1,0.00,0.13,-0.04,0.01,13.53,17.90,0.756,207.57,34.65,7.70,9.36,0.822,3.95,5.41,0.731,1.25,1.66,0.750,1.14,1.04,0.31,0.00,0.94,17.48,0.42,0.00,0.31,5.51,0.21,0.31,0.00,...,0.239,0.31,4.27,4.99,0.52,0.31,0.0,0.21,0.94,0.42,0.0,29.13,1.14,3.33,16.65,11.45,2.71,28.51,1.04,1.98,0.526,1.25,0.00,18.94,77.62,37.98,2.81,1.14,0.83,3.23,1.98,48.90,23.10,0.472,5.51,0.0,2.39,1.14,0.21,0.00,0.0,0.00,3.12,2.60,3.43,0.431,2.2,0,1,0
4,24,AarÃ³n MartÃ­n,0.00,0.11,0.00,0.00,0.00,0.11,0.00,0.02,0.02,0.10,0.11,0.54,0.07,0.133,0.00,0.00,23.1,0.04,0.03,-0.02,-0.02,37.26,48.72,0.765,656.88,231.19,17.70,19.59,0.904,14.69,18.07,0.813,4.06,9.29,0.438,1.02,1.96,0.65,0.44,2

# v9: Rearrange position attribute to start

In [256]:
import copy
cols = list(df.columns.values)
cols.insert(1, cols.pop())
cols.insert(1, cols.pop())
cols.insert(3, cols.pop())
new_attr_order = copy.copy(cols)
df=df[new_attr_order]
df.to_csv("dataset_v9.csv")
df

,Age,Foward,Midfielder,Defender,Name,Goals,Assists,Non-Penalty Goals,Penalty Kicks Made,Penalty Kicks Attempted,Yellow Cards,Red Cards,xG,npxG,xA,npxG+xA,Shots Total,Shots on target,Shots on target %,Goals/Shot,Goals/Shot on target,Average Shot Distance,Shots from free kicks,npxG/Sh,Goals - xG,Non-Penalty Goals - npxG,Passes Completed,Passes Attempted,Pass Completion %,Total Passing Distance,Progressive Passing Distance,Passes Completed (Short),Passes Attempted (Short),Pass Completion % (Short),Passes Completed (Medium),Passes Attempted (Medium),Pass Completion % (Medium),Passes Completed (Long),Passes Attempted (Long),Pass Completion % (Long),Key Passes,Passes into Final Third,Passes into Penalty Area,Crosses into Penalty Area,Progressive Passes,Live-ball passes,Dead-ball passes,Passes from Free Kicks,Through Balls,Passes Under Pressure,...,Dribbled Past,Pressures,Successful Pressures,Successful Pressure %,Pressures (Def 3rd),Pressures (Mid 3rd),Pressures (Att 3rd),Blocks,Shots Blocked,Shots Saved,Interceptions,Tkl+Int,Clearances,Errors,Touches,Touches (Def Pen),Touches (Def 3rd),Touches (Mid 3rd),Touches (Att 3rd),Touches (Att Pen),Touches (Live-Ball),Dribbles Completed,Dribbles Attempted,Successful Dribble %,Players Dribbled Past,Nutmegs,Carries,Total Carrying Distance,Progressive Carrying Distance,Progressive Carries,Carries into Final Third,Carries into Penalty Area,Miscontrols,Dispossessed,Pass Targets,Passes Received,Passes Received %,Progressive Passes Rec,Second Yellow Card,Fouls Committed,Fouls Drawn,Offsides,Penalty Kicks Won,Penalty Kicks Conceded,Own Goals,Ball Recoveries,Aerials won,Aerials lost,% of Aerials Won,Market value
0,21,1,0,0,Aaron Connolly,0.19,0.00,0.19,0.00,0.00,0.00,0.00,0.54,0.54,0.08,0.61,3.06,0.96,0.313,0.06,0.20,13.4,0.00,0.17,-0.34,-0.34,8.23,10.53,0.782,113.94,8.81,5.17,6.70,0.771,2.68,3.06,0.875,0.19,0.19,1.000,0.57,0.00,0.00,0.00,0.00,9.77,0.77,0.00,0.00,1.91,...,0.57,13.40,4.21,0.314,0.77,6.51,6.13,1.91,0.19,0.0,0.00,0.96,0.19,0.0,22.40,0.19,1.91,9.00,13.02,4.40,21.64,0.38,0.57,0.667,0.38,0.00,11.87,71.81,41.94,2.30,0.96,0.57,2.49,0.96,40.98,15.13,0.369,6.70,0.0,0.38,2.30,0.19,0.00,0.0,0.00,2.87,0.96,3.26,0.227,7.0
1,31,0,0,1,Aaron Cresswell,0.02,0.20,0.02,0.00,0.00,0.07,0.00,0.03,0.03,0.16,0.20,0.62,0.10,0.160,0.04,0.25,22.8,0.32,0.05,-0.01,-0.01,49.11,63.91,0.768,929.47,423.76,21.52,23.98,0.897,20.24,23.80,0.850,6.67,13.81,0.483,1.67,4.11,0.96,0.37,4.68,50.71,13.19,2.26,0.02,5.49,...,0.52,6.35,2.14,0.337,3.27,2.46,0.62,1.43,0.42,0.0,1.21,2.09,2.49,0.0,71.27,4.28,21.29,32.42,21.64,0.76,58.05,0.15,0.44,0.333,0.15,0.00,40.64,185.56,94.16,3.64,1.62,0.12,0.27,0.32,44.43,42.61,0.959,1.06,0.0,0.52,0.44,0.02,0.00,0.0,0.00,8.49,1.16,0.69,0.627,5.0
2,19,0,0,1,Aaron Hickey,0.21,0.00,0.21,0.00,0.00,0.28,0.07,0.03,0.03,0.02,0.05,0.77,0.42,0.545,0.27,0.50,23.6,0.00,0.03,0.18,0.18,37.27,44.37,0.840,597.47,155.69,19.02,20.69,0.919,15.33,18.18,0.843,2.37,4.18,0.567,0.49,1.18,0.28,0.07,1.81,35.67,8.71,0.98,0.00,6.76,...,0.35,13.37,3.69,0.276,6.41,4.32,2.65,2.30,0.56,0.0,1.04,2.51,1.60,0.0,55.10,3.07,17.69,24.80,16.02,0.84,46.74,1.32,2.09,0.633,1.53,0.07,35.94,196.58,106.93,4.74,0.91,0.35,0.91,0.84,37.89,33.37,0.881,2.51,0.0,1.32,2.23,0.00,0.00,0.0,0.00,5.99,0.63,0.84,0.429,8.0
3,24,1,0,0,Aaron Leya Iseka,0.31,0.00,0.21,0.10,0.21,0.42,0.00,0.35,0.20,0.07,0.27,1.56,0.52,0.333,0.13,0.40,15.1,0.00,0.13,-0.04,0.01,13.53,17.90,0.756,207.57,34.65,7.70,9.36,0.822,3.95,5.41,0.731,1.25,1.66,0.750,1.14,1.04,0.31,0.00,0.94,17.48,0.42,0.00,0.31,5.51,...,0.73,9.57,2.29,0.239,0.31,4.27,4.99,0.52,0.31,0.0,0.21,0.94,0.42,0.0,29.13,1.14,3.33,16.65,11.45,2.71,28.51,1.04,1.98,0.526,1.25,0.00,18.94,77.62,37.98,2.81,1.14,0.83,3.23,1.98,48.90,23.10,0.472,5.51,0.0,2.39,1.14,0.21,0.00,0.0,0.00,3.12,2.60,3.43,0.431,2.2
4,24,0,0,1,AarÃ³n MartÃ­n,0.00,0.11,0.00,0.00,0.00,0.11,0.00,0.02,0.02,0.10,0.11,0.54,0.07,0.133,0.00,0.00,23.1,0.04,0.03,-0.02,-0.02,37.26,48.72,0.765,656.88,231.19,17.70,19.59,0.904,14.69,18.07,0.813,4.06,9.29,0.43

# v10: Drop name

In [257]:
df = pd.read_csv("dataset_v9.csv",index_col=[0])
df = df.drop("Name",axis=1)
df.to_csv("dataset_v10.csv")
df

,Age,Foward,Midfielder,Defender,Goals,Assists,Non-Penalty Goals,Penalty Kicks Made,Penalty Kicks Attempted,Yellow Cards,Red Cards,xG,npxG,xA,npxG+xA,Shots Total,Shots on target,Shots on target %,Goals/Shot,Goals/Shot on target,Average Shot Distance,Shots from free kicks,npxG/Sh,Goals - xG,Non-Penalty Goals - npxG,Passes Completed,Passes Attempted,Pass Completion %,Total Passing Distance,Progressive Passing Distance,Passes Completed (Short),Passes Attempted (Short),Pass Completion % (Short),Passes Completed (Medium),Passes Attempted (Medium),Pass Completion % (Medium),Passes Completed (Long),Passes Attempted (Long),Pass Completion % (Long),Key Passes,Passes into Final Third,Passes into Penalty Area,Crosses into Penalty Area,Progressive Passes,Live-ball passes,Dead-ball passes,Passes from Free Kicks,Through Balls,Passes Under Pressure,Switches,...,Dribbled Past,Pressures,Successful Pressures,Successful Pressure %,Pressures (Def 3rd),Pressures (Mid 3rd),Pressures (Att 3rd),Blocks,Shots Blocked,Shots Saved,Interceptions,Tkl+Int,Clearances,Errors,Touches,Touches (Def Pen),Touches (Def 3rd),Touches (Mid 3rd),Touches (Att 3rd),Touches (Att Pen),Touches (Live-Ball),Dribbles Completed,Dribbles Attempted,Successful Dribble %,Players Dribbled Past,Nutmegs,Carries,Total Carrying Distance,Progressive Carrying Distance,Progressive Carries,Carries into Final Third,Carries into Penalty Area,Miscontrols,Dispossessed,Pass Targets,Passes Received,Passes Received %,Progressive Passes Rec,Second Yellow Card,Fouls Committed,Fouls Drawn,Offsides,Penalty Kicks Won,Penalty Kicks Conceded,Own Goals,Ball Recoveries,Aerials won,Aerials lost,% of Aerials Won,Market value
0,21,1,0,0,0.19,0.00,0.19,0.00,0.00,0.00,0.00,0.54,0.54,0.08,0.61,3.06,0.96,0.313,0.06,0.20,13.4,0.00,0.17,-0.34,-0.34,8.23,10.53,0.782,113.94,8.81,5.17,6.70,0.771,2.68,3.06,0.875,0.19,0.19,1.000,0.57,0.00,0.00,0.00,0.00,9.77,0.77,0.00,0.00,1.91,0.00,...,0.57,13.40,4.21,0.314,0.77,6.51,6.13,1.91,0.19,0.0,0.00,0.96,0.19,0.0,22.40,0.19,1.91,9.00,13.02,4.40,21.64,0.38,0.57,0.667,0.38,0.00,11.87,71.81,41.94,2.30,0.96,0.57,2.49,0.96,40.98,15.13,0.369,6.70,0.0,0.38,2.30,0.19,0.00,0.0,0.00,2.87,0.96,3.26,0.227,7.0
1,31,0,0,1,0.02,0.20,0.02,0.00,0.00,0.07,0.00,0.03,0.03,0.16,0.20,0.62,0.10,0.160,0.04,0.25,22.8,0.32,0.05,-0.01,-0.01,49.11,63.91,0.768,929.47,423.76,21.52,23.98,0.897,20.24,23.80,0.850,6.67,13.81,0.483,1.67,4.11,0.96,0.37,4.68,50.71,13.19,2.26,0.02,5.49,1.87,...,0.52,6.35,2.14,0.337,3.27,2.46,0.62,1.43,0.42,0.0,1.21,2.09,2.49,0.0,71.27,4.28,21.29,32.42,21.64,0.76,58.05,0.15,0.44,0.333,0.15,0.00,40.64,185.56,94.16,3.64,1.62,0.12,0.27,0.32,44.43,42.61,0.959,1.06,0.0,0.52,0.44,0.02,0.00,0.0,0.00,8.49,1.16,0.69,0.627,5.0
2,19,0,0,1,0.21,0.00,0.21,0.00,0.00,0.28,0.07,0.03,0.03,0.02,0.05,0.77,0.42,0.545,0.27,0.50,23.6,0.00,0.03,0.18,0.18,37.27,44.37,0.840,597.47,155.69,19.02,20.69,0.919,15.33,18.18,0.843,2.37,4.18,0.567,0.49,1.18,0.28,0.07,1.81,35.67,8.71,0.98,0.00,6.76,0.42,...,0.35,13.37,3.69,0.276,6.41,4.32,2.65,2.30,0.56,0.0,1.04,2.51,1.60,0.0,55.10,3.07,17.69,24.80,16.02,0.84,46.74,1.32,2.09,0.633,1.53,0.07,35.94,196.58,106.93,4.74,0.91,0.35,0.91,0.84,37.89,33.37,0.881,2.51,0.0,1.32,2.23,0.00,0.00,0.0,0.00,5.99,0.63,0.84,0.429,8.0
3,24,1,0,0,0.31,0.00,0.21,0.10,0.21,0.42,0.00,0.35,0.20,0.07,0.27,1.56,0.52,0.333,0.13,0.40,15.1,0.00,0.13,-0.04,0.01,13.53,17.90,0.756,207.57,34.65,7.70,9.36,0.822,3.95,5.41,0.731,1.25,1.66,0.750,1.14,1.04,0.31,0.00,0.94,17.48,0.42,0.00,0.31,5.51,0.21,...,0.73,9.57,2.29,0.239,0.31,4.27,4.99,0.52,0.31,0.0,0.21,0.94,0.42,0.0,29.13,1.14,3.33,16.65,11.45,2.71,28.51,1.04,1.98,0.526,1.25,0.00,18.94,77.62,37.98,2.81,1.14,0.83,3.23,1.98,48.90,23.10,0.472,5.51,0.0,2.39,1.14,0.21,0.00,0.0,0.00,3.12,2.60,3.43,0.431,2.2
4,24,0,0,1,0.00,0.11,0.00,0.00,0.00,0.11,0.00,0.02,0.02,0.10,0.11,0.54,0.07,0.133,0.00,0.00,23.1,0.04,0.03,-0.02,-0.02,37.26,48.72,0.765,656.88,231.19,17.70,19.59,0.904,14.69,18.07,0.813,4.06,9.29,0.438,1.02,1.96,0.65,0.44,2.50,38.31,10.41,1.20,0.07,4.4

In [258]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1894 entries, 0 to 1893
Data columns (total 139 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Age                            1894 non-null   int64  
 1   Foward                         1894 non-null   int64  
 2   Midfielder                     1894 non-null   int64  
 3   Defender                       1894 non-null   int64  
 4   Goals                          1894 non-null   float64
 5   Assists                        1894 non-null   float64
 6   Non-Penalty Goals              1894 non-null   float64
 7   Penalty Kicks Made             1894 non-null   float64
 8   Penalty Kicks Attempted        1894 non-null   float64
 9   Yellow Cards                   1894 non-null   float64
 10  Red Cards                      1894 non-null   float64
 11  xG                             1894 non-null   float64
 12  npxG                           1894 non-null   